# Importowanie bibliotek
- *subprocess* - uruchamianie programów zewnętrznych w skrypcie, uzyskiwanie z nich danych
- *numpy* - obliczenia macierzowe, działania na macierzach
- *matplotlib.pyplot* - biblioteka do rysowania wykresów w stylu MatLab'a

In [1]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import custom_statistics as stat

# Uzyskiwanie wyników pomiaru z programu

In [2]:
X = np.linspace(0, 20, 20)
U = np.zeros(X.size)

for i, val in enumerate(X):
	program_output = subprocess.run(["z09v01.exe", str(val)], capture_output=True, text=True)
	U[i] = program_output.stdout


In [3]:
# plt.plot(x, t, 'ro')

# Standaryzacja pomiarów do przedziału <-1; 1>
W kolejności obliczamy wartość średnią i jednostkę zmienności:
$$\bar{x}=\frac{x_{i,min} + x_{i,max}}{2}$$
$$\varDelta x=\frac{x_{i,max} + x_{i.min}}{2}$$
Następnie przeliczamy wartości $x_i$ do nowego ukladu współrzędnych:
$$t_i = \frac{x_i - \bar{x}}{\varDelta x}$$
gdzie $i = 1,2,...,N$

In [4]:
average_x = (X[0] + X[-1]) / (2)
delta_x   = (X[-1]+ X[0] ) / (2)

T = [(x - average_x)/delta_x for x in X]

In [5]:
def min_value(arr: np.ndarray):
	min = arr[0]
	for i in arr: 
		if i < min: min = i

	return min

In [6]:
print(stat.min_value(U))
print(min_value(U))
min = U[0]
for i in U: 
	if i < min: min = i

print(min)


11.86311279
11.86311279
11.86311279
